In [57]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [58]:
#load dataset
train= pd.read_csv('train.csv')
test= pd.read_csv('test.csv')
test_ids=test["PassengerId"]

In [59]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [60]:
#Understand the nature of the data
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [61]:
#Comparing survival rate across Age, SibSp, Parch, Fare
print(pd.pivot_table(train, index = "Survived", values=["Age", "SibSp", "Parch", "Fare"]))
print()

                Age       Fare     Parch     SibSp
Survived                                          
0         30.626179  22.117887  0.329690  0.553734
1         28.343690  48.395408  0.464912  0.473684



In [62]:
#Comparing survival rate of each of categorical values
print(pd.pivot_table(train, index = "Survived", columns= 'Pclass', values ="Ticket", aggfunc="count"))
print()
print(pd.pivot_table(train, index = "Survived", columns= 'Sex', values ="Ticket", aggfunc="count"))
print()
print(pd.pivot_table(train, index = "Survived", columns= 'Embarked', values ="Ticket", aggfunc="count"))
print()

Pclass      1   2    3
Survived              
0          80  97  372
1         136  87  119

Sex       female  male
Survived              
0             81   468
1            233   109

Embarked   C   Q    S
Survived             
0         75  47  427
1         93  30  217



In [63]:
#missing values
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [64]:
#Data Cleaning
def clean(data):
    data= data.drop(["Ticket", "Name", "PassengerId", "Cabin"], axis=1)
    data.Embarked.fillna("U", inplace=True)
    data.interpolate(limit_direction="both",inplace=True)
    return data
train= clean(train)
test= clean(test)

In [65]:
#Initialize LabelEncoder
# Fit LabelEncoder to the data and transform data to numeric labels
data = preprocessing.LabelEncoder()
columns=["Sex", "Embarked"]
for col in columns:
    train[col]= data.fit_transform(train[col])
    test[col]= data.fit_transform(test[col])
    print(data.classes_)
    
train.head()

['female' 'male']
['C' 'Q' 'S']


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [66]:
# Split the dataset into training and testing sets
y= train["Survived"]
x=train.drop("Survived", axis=1)

x_train, x_val, y_train, y_val =train_test_split(x, y, test_size=0.2, random_state=42)

In [67]:
# Train and evaluate Logistic Regression model
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(x_train, y_train)
logistic_pred = logistic_model.predict(x_val)
logistic_accuracy = accuracy_score(y_val, logistic_pred)
print("Logistic Regression Accuracy:", logistic_accuracy)

Logistic Regression Accuracy: 0.8100558659217877


In [68]:
# Make predictions on the test set
submission_preds = logistic_model.predict(test)

In [69]:
# Prepare submission file
df = pd.DataFrame({"PassengerId":test_ids.values,
                  "Survived":submission_preds,
                  })

In [70]:
df.to_csv("submission.csv", index=False)